# Modeling

In [1]:
import sqlite3 as sq3
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')

import config
import matplotlib.pyplot as plt
%matplotlib inline

conn = sq3.connect(config.mlb_db_path)
event = pd.read_sql(sql='select * from bevent where gameid like "%201%"', con=conn)

event['date'] = event['gameid'].apply(lambda x: int(x[3:]))
cut_point = 201707060
learn, test = event[event['date'] < cut_point], event[event['date'] >= cut_point]
print('Test/Learn Ratio: {}%'.format(round(float(len(test))/float(len(learn)),4)*100))

Test/Learn Ratio: 19.62%


# Pitcher History

In [2]:
test_ = test[(test['date'] == 201708060)]# & (test['gameid'] == "LAN201707060")]
set(test_['gameid'])

{u'ANA201708060',
 u'ATL201708060',
 u'BAL201708060',
 u'BOS201708060',
 u'CHN201708060',
 u'CIN201708060',
 u'CLE201708060',
 u'COL201708060',
 u'HOU201708060',
 u'MIN201708060',
 u'NYN201708060',
 u'PIT201708060',
 u'SFN201708060',
 u'TBA201708060'}

In [14]:
test_ = test[(test['gameid'] == "BAL201708060")]

# Set pitchers
gspec = test_[['inning','battingteam','vteam','pitcher']].groupby(['battingteam','pitcher']).agg({'inning':'min'})
t0_p = list(gspec[gspec['inning']==1].loc[0].index)
t1_p = list(gspec[gspec['inning']==1].loc[1].index)

# find starting roster for each team
t0_ros = list(test_[test_['battingteam'] == 0].groupby('batter').count(
    ).sort_values('gameid', ascending=False).index[:9])
t1_ros = list(test_[test_['battingteam'] == 1].groupby('batter').count(
    ).sort_values('gameid', ascending=False).index[:9])

#Slice learn dataset to get previous stats vs SP
t0 = learn[(learn['batter'].isin(t0_ros)) & (learn['pitcher'].isin(t1_p))]
t1 = learn[(learn['batter'].isin(t1_ros)) & (learn['pitcher'].isin(t0_p))]

x0 = {'BA':(t0['hitvalue']/t0['hitvalue']).fillna(0).sum()/len(t0),
 'SLG':float((t0['hitvalue']).sum())/len(t0)}
x1 = {'BA':(t1['hitvalue']/t1['hitvalue']).fillna(0).sum()/len(t1),
 'SLG':float((t1['hitvalue']).sum())/len(t1)}
print('t0',x0, len(t0))
print('t1',x1, len(t1))

('t0', {'SLG': 0.45454545454545453, 'BA': 0.22727272727272727}, 22)
('t1', {'SLG': 0.2631578947368421, 'BA': 0.18421052631578946}, 38)


In [15]:
test_['hitvalue_'] = (test_['hitvalue']/test_['hitvalue']).fillna(0)
ta_ = test_.groupby('battingteam').agg({'hitvalue_':'sum',
                                        'hitvalue':'sum',
                                 'gameid':'count'})
ta_['BA'] = ta_['hitvalue_']/ta_['gameid']
ta_['SLG'] = ta_['hitvalue']/ta_['gameid']

ta_[['BA','SLG']]

,BA,SLG
battingteam,,
0,0.318182,0.431818
1,0.377778,0.777778


In [10]:
test_[-1:][['vscore','hscore']]

,vscore,hscore
1361366,3,12
